In [3]:
import requests
from bs4 import BeautifulSoup
from typing import List, Dict

from serpapi import GoogleSearch

from dotenv import load_dotenv
import os

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
import json

from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import Document

from langchain.text_splitter import CharacterTextSplitter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [4]:
print("Current working directory:", os.getcwd())
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print('OPEN API KEY FOUND')
else:
    print("No API key found")

serp_api_key = os.getenv("SERP_API_KEY")
if serp_api_key:
    print('SERP API KEY FOUND')
else:
    print("No SERP API key found")

sarvam_api_key = os.getenv("SARVAM_API_KEY")
if sarvam_api_key:
    print('SARVAM API KEY FOUND')
else:
    print("No SARVAM API key found")

grok_api_key = os.getenv("GROK_API_KEY")
if grok_api_key:
    print('GROK API KEY FOUND')
else:
    print("No GROK API key found")

serp_dev_api_key = os.getenv("SERP_DEV_API_KEY")
if serp_dev_api_key:
    print('SERP DEV API KEY FOUND')
else:
    print("No SERP API key found")

tav_dev_api_key = os.getenv("TAVILY_API_KEY")
if tav_dev_api_key:
    print('TAV DEV API KEY FOUND')
else:
    print("No TAV API key found")

Current working directory: c:\Users\rahul\OneDrive\7_Learning\IISC\Courses\3.1_Deep_Learning\Course Material\Project\wip
OPEN API KEY FOUND
SERP API KEY FOUND
SARVAM API KEY FOUND
GROK API KEY FOUND
SERP DEV API KEY FOUND
TAV DEV API KEY FOUND


In [5]:
class SerperRetrieverWrapper:
    def __init__(self, api_key: str, num_results: int = 5):
        self.api_key = api_key
        self.num_results = num_results
    
    def get_relevant_documents(self, query: str):
        """
        Query Serper.dev and return up to `num_results` organic search hits.
        Each hit is a dict: { "title": str, "link": str, "snippet": str }.
        Raises HTTPError on non-2xx responses.
        """
        _SERPER_SEARCH_URL = "https://google.serper.dev/search"
    
        headers = {
            "X-API-KEY": self.api_key,
            "Content-Type": "application/json"
        }
        payload = {
            "q": 'site:news18.com ' + query,
            "num": self.num_results,
        }
    
        resp = requests.post(_SERPER_SEARCH_URL, headers=headers, json=payload, timeout=5)

        if resp.status_code != 200:
            raise Exception(f"Serper API Error: {resp.text}")
        results = resp.json()
    
        documents = []
        for result in results.get("organic", [])[:self.num_results]:
            content = f"{result.get('title', '')}\n{result.get('snippet', '')}"
            documents.append(Document(page_content=content, metadata={"source": result.get("link", "")}))
    
        return documents

In [6]:
serper_retriever = SerperRetrieverWrapper(api_key=serp_dev_api_key)
context_retriever = RunnableLambda(serper_retriever.get_relevant_documents)

In [7]:
prompt = PromptTemplate.from_template("""
You are a fake news detection assistant. Do not include any preamble or commentary. 
Based on the context from reliable sources, your task is to classify a claim as either REAL or FAKE.

You will be given:
    - A claim
    - Context from reliable sources

Your task is:
    1. Analyze the claim using the provided evidence.
    2. Classify the claim as REAL or FAKE.
    3. Provide a brief explanation for your classification.
    4. Output your answer in the following format:
    {{
        "Classification": "REAL" or "FAKE",
        "Explanation": "<1-2 sentence justification>"
    }}

claim: {question}

Context:
{context}
""")

In [8]:
summarizer_prompt = PromptTemplate.from_template("""
You are an assistant summarizing factual evidence from multiple documents.

Based on the following documents, extract the key facts relevant to the claim.

Claim: {question}

Documents:
{context}

Return a short neutral summary of the key facts only.
""")

In [9]:
llm1 = ChatGroq(api_key=grok_api_key, model_name="llama3-8b-8192")
summarizer_chain = (
    {
        "context": context_retriever,
        "question": RunnablePassthrough()
    }
    | summarizer_prompt
    | llm1
    | StrOutputParser()
)

In [10]:
judge_prompt = PromptTemplate.from_template("""
You are a fact-checking assistant.

Claim: {question}

Evidence:
{evidence}

Decide whether the claim is REAL or FAKE or UNSURE based only on the evidence.

Respond in this format:
Classification: REAL or FAKE
Explanation: <your reasoning>
""")

In [11]:
llm2 = ChatGroq(api_key=grok_api_key, model_name="llama3-8b-8192")
fact_checker_chain = (
    {
        "question": RunnablePassthrough(),
        "evidence": summarizer_chain 
    }
    | judge_prompt
    | llm2
    | StrOutputParser()
)

In [17]:
claim = "australia beats south africa in the test series final"

In [18]:
fact_checker_chain.invoke(claim)

'Classification: FAKE\nExplanation: The evidence provided shows that South Africa actually beat Australia in the ICC World Test Championship 2025 final, which contradicts the claim that Australia beats South Africa in the test series final. Therefore, the claim is false and classified as FAKE.'

In [19]:
summarizer_chain.invoke(claim)

'Based on the provided documents, the key facts relevant to the claim "Australia beats South Africa in the test series final" are:\n\n* The claim is actually false, as the documents confirm that South Africa won the ICC World Test Championship final against Australia.\n* The match was held at Lord\'s, and South Africa won by five wickets on the fourth day.\n* This marks South Africa\'s first major title in 27 years.\n* Australian captain Temba Bavuma revealed that the Australian team used sledging tactics, calling them "chokers".\n\nThese key facts are based on the content of the provided documents, which include news articles and live score updates from the ICC World Test Championship final.'